# GPU-accelerated molecule characterization and clustering

This notebook demonstrates a molecular clustering example. We will perform the same workflow with both RDKit and nvMolKit APIs. The steps are:

1. Compute Morgan Fingerprints for molecules
2. Compute tanimoto similarity between fingerprints
3. Cluster molecules via Butina clustering using tanimoto distances as clustering metrics

## User parameters

In [ ]:
# Number of molecules to process
n_mols = 10000
# Hardware threads to use
n_cpu_threads = 16

# Radius for Morgan fingerprinting
fp_radius = 3
# Number of fingerprint bits
fp_nbits = 1024
# Butina similarity threshold
distance_threshold = 0.5

## Molecule selection

First we'll parse molecules using RDKit. If you have a .smi file, use this section to load molecules, replace smi_file with a local path. The default path will load molecules store in the nvmolkit repo.


In [ ]:
import pandas as pd
from rdkit.Chem import MolFromSmiles

smi_file = "../benchmarks/data/chembl_10k.smi"

smis = pd.read_csv(smi_file, nrows=n_mols).iloc[:, 0].to_list()
mols = [MolFromSmiles(smi) for smi in smis]
mols = [mol for mol in mols if mol] # remove any parse failures
n_mols = len(mols)
print(f"Parsed {n_mols} mols")



## RDKit workflow

The following lines compute butina clusters via RDKit. The fingerprinting API is multithreaded, but parallelizing the similarity and clustering would require user-defined multiprocessing, which has significant overhead. See the appendix of this notebook for an example.

In [ ]:
import time

from rdkit.Chem import rdFingerprintGenerator
from rdkit.DataStructs import BulkTanimotoSimilarity
from rdkit.ML.Cluster.Butina import ClusterData

t = time.time()
print("Computing fingerprints")
generator = rdFingerprintGenerator.GetMorganGenerator(radius=fp_radius, fpSize=fp_nbits)
fps = generator.GetFingerprints(mols, numThreads=n_cpu_threads)

print("Computing cross tanimoto distances")
distances = []
for i in range(len(mols)):
    distances.extend(BulkTanimotoSimilarity(fps[i], fps[:i], returnDistance=True))

print("Computing clusters")
clusters = ClusterData(
    distances,
    n_mols,
    distance_threshold,
    isDistData=True,
    distFunc=None,
    reordering=True
)

print(f"Total elapsed time: {time.time() - t:.2f} seconds")

## nvMolKit workflow


In [ ]:
import time
import torch

from nvmolkit.similarity import crossTanimotoSimilarity
from nvmolkit.fingerprints import MorganFingerprintGenerator

t = time.time()
nvmolkit_fpgen = MorganFingerprintGenerator(radius=fp_radius, fpSize=fp_nbits)
# Note that nvmolkit fingerprints use multithreading for preprocessing in addition to GPU acceleration
nvmolkit_fps_cu = torch.as_tensor(nvmolkit_fpgen.GetFingerprints(mols, num_threads=n_cpu_threads), device='cuda')
nvmolkit_distances = 1.0 - torch.as_tensor(crossTanimotoSimilarity(nvmolkit_fps_cu),device='cuda')
torch.cuda.synchronize()
nvmolkit_clusters = ClusterData(
    nvmolkit_distances.cpu().numpy(),
    n_mols,
    distance_threshold,
    isDistData=True,
    distFunc=None,
    reordering=True
)
print(f"Total elapsed time: {time.time() - t:.2f} seconds")

## Comparing results

We'll plot the sizes of the resulting clusters to compare between the two methods

In [ ]:
import matplotlib.pyplot as plt

cluster_counts = [len(item) for item in clusters]
nvmolkit_cluster_counts = [len(item) for item in nvmolkit_clusters]

plt.plot(range(len(cluster_counts)), cluster_counts, label="RDKit", linewidth=3)
plt.plot(range(len(nvmolkit_cluster_counts)), nvmolkit_cluster_counts, linestyle="--", linewidth=3, label="nvMolKit")
plt.xlabel("Cluster")
plt.legend()
plt.ylabel("Number of Molecules")
plt.ylim(0, max(cluster_counts) * 1.05)


## Appendix - RDKit workflow with multiprocessing

The following lines compute butina clusters via RDKit, with some multiprocess acceleration

In [ ]:
import multiprocessing

CHUNK_SIZE = 500

def fps_distributed(mols):
    """Generates Morgan fingerprints for a list of molecules."""
    generator = rdFingerprintGenerator.GetMorganGenerator(radius=fp_radius, fpSize=fp_nbits)
    return [generator.GetFingerprint(mol) for mol in mols]

def distances_distributed(fp0, fp_slice):
    """Computes the Tanimoto distances between one and a set of fingerprints."""
    return BulkTanimotoSimilarity(fp0, fp_slice, returnDistance=True)

t = time.time()
print("Computing fingerprints")
with multiprocessing.Pool(n_cpu_threads) as pool:
    chunked_mols = [mols[i: i + CHUNK_SIZE] for i in range(0, len(mols), CHUNK_SIZE)]
    chunked_fps = pool.map(fps_distributed, chunked_mols)
    fps = []
    for chunk in chunked_fps:
        fps.extend(chunk)

    chunked_distance_input = [(fps[i], fps[:i]) for i in range(n_mols)]
    chunked_distances = pool.starmap(distances_distributed, chunked_distance_input)
    distances = []
    for chunk in chunked_distances:
        distances.extend(chunk)

print("Computing clusters")
clusters = ClusterData(
    distances,
    n_mols,
    distance_threshold,
    isDistData=True,
    distFunc=None,
    reordering=True
)

print(f"Total elapsed time: {time.time() - t:.2f} seconds")